In [1]:
import chart_studio.plotly as py
import cufflinks as cf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')

plt.style.use('ggplot')

cf.go_offline()

In [2]:
query = """
SELECT 
    DATETIME_TRUNC(pickup_datetime, hour) as pickup_hour,
    count(*) as cnt
FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015` 
WHERE EXTRACT(MONTH from pickup_datetime) = 1
GROUP BY pickup_hour
ORDER BY pickup_hour
"""

df = pd.read_gbq(query= query, dialect='standard', project_id='new-york-taxi-309501', auth_local_webserver=True)

Downloading: 100%|████████████████████████████████████████████████████████████████| 744/744 [00:01<00:00, 622.31rows/s]


* 구글 빅쿼리( Google BigGuery )에서 SQL언어를 사용해 데이터를 추출
* 문법은 'Standard'을 사용
* 추출한 데이터를 판다스 데이터프레임으로 만들어주는 함수 ( read_gbq () ) 사용

In [3]:
df.head(10)

,pickup_hour,cnt
0,2015-01-01 00:00:00,28312
1,2015-01-01 01:00:00,31707
2,2015-01-01 02:00:00,28068
3,2015-01-01 03:00:00,24288
4,2015-01-01 04:00:00,17081
5,2015-01-01 05:00:00,9112
6,2015-01-01 06:00:00,6637
7,2015-01-01 07:00:00,6011
8,2015-01-01 08:00:00,5586
9,2015-01-01 09:00:00,6977


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 744 entries, 0 to 743
Data columns (total 2 columns):
pickup_hour    744 non-null datetime64[ns]
cnt            744 non-null int64
dtypes: datetime64[ns](1), int64(1)
memory usage: 11.8 KB


In [5]:
df = df.set_index('pickup_hour')
df.head(10)

,cnt
pickup_hour,
2015-01-01 00:00:00,28312
2015-01-01 01:00:00,31707
2015-01-01 02:00:00,28068
2015-01-01 03:00:00,24288
2015-01-01 04:00:00,17081
2015-01-01 05:00:00,9112
2015-01-01 06:00:00,6637
2015-01-01 07:00:00,6011
2015-01-01 08:00:00,5586
